## ST449 Research Project Report

# *Creating a End-to-End Task-Completion Chatbot using Deep Q-Network*

## 0. Project Overview
With increasing exposure to advanced technology in our day-to-day lives, we live in the era of the Artificial Intelligence. A part of this new Artificial Intelligence development is Dialogue Systems also know as Chatbots, which usage is increasing every day. Examples of such systems are Apple's Siri, Amazon's Alexa and the Google Assistant. Goal-Oriented dialogue system is a specfic type of Chatbot which helps users in achieving a predefined goal within a closed domain. A closed domain dialogue system focuses only on several pre-identified features and have predetermined intents, with a purpose to help the user to easily achieve some goal. In our case, we are interested in a Goal-Oriented Chatbot that could help users to find and book movie tickets through conversation-like interactions.

The project aims to build a Deep Q-Network Chatbot dialogue manager model and a User Simuator system to implement a Reinforcement Learning pipeline for training the Chatbot agent with a Deep Q-Network and with 2 Reinforcement Learning improvements namely the Double Deep Q-Network (Hasselt et al., 2015) and Prioritized Experience Replay (Schaul et al., 2015). Our results are evaluated using the average reward per 100 episodes and the best success rate. 

<i>This project includes:</i>

- Implementing the Dialogue Manager system inside the End-to-End Dialogue System exclusively which includes the neural network Agent and the Dialogue State Tracker, the User Simulator and the Error Model Controller.
- Training our model with a Deep Q-Network, and examine how supplements to the Reinforcement Learning network such as the Double Deep Q-Network and Prioritized Experience Replay in the context of a Dialogue System improve its accuracy, efficiency and stability.
- Discussions on further research and improvement areas in the Dialogue Manager system.

<i>Artificial Intelligence and Deep Learning concepts in this project including further research ideas:</i>

- Theorized how to supplement the DQN Model with Natural Language Processing Units in order to understand human-level conversations

#### 0.1 Summary of Results

We successfully implemented the DQN Chatbot Agent and compare the original Model to the DDQN Agent and the DDQN+PER Agent. Overall, the latter 2 perform better in term of both a success rates and average reward as well as the stability in performance. This is shown by the tighter standard deviations of the sucess rates and average reward in the DDQN Agent and DDQN+PER Agent comparing to the original Agent. 

Following table summarizes the results for the DQN Agent benchmark


|                | DQN               |       DDQN       | DDQN+PER         |
|----------------|-------------------|:----------------:|------------------|
| Success Rate   |  100%             |              107%|             110% |
| Average Reward |  100%             |              113%|              119%|



## 1. Introduction and Research Question

##### 1.1 Research Goals and Question

This report aims to implement and evaluate the Deep Q-Network (DQN) Chatbot Dialogue Manager (DM) system as detailed in the paper "End-to-End Task-Completion Neural Dialogue Systems" by Li et al. (2017) against 2 other Deep Q-Learning Reinforcement Learning (RL) improvements. As the Natural Language Generator and Natural Language Understanding systems are not included, our results are incomparable to that of the original paper. We include the User Simulator module for training our Chatbot. The goal of this report is to implement the system and RL approaches correctly and compare their performances in terms of average rewards, sucess rates and stability during learning.

##### 1.2 Artificial Intelligence and Deep Learning Research Context

In Deep Reinforcement Learning, the success of the DQN for games has inspired similar approaches in other domains. Whilst Goal-Oriented (GO) dialogue systems remain to be the most prominent component in today’s virtual personal assistants due to its ability to understand natural language user utterance and accomplish tasks more efficiently, there are areas of improvement in terms of capturing the natural dynamics of human conversations. Rudnicky et al. pioneered a Dialogue System pipeline consisting of a Language Understanding (LU) module, a Dialogue Manager (DM), and a Natural Language Generation (NLG) component. However, the model was complex and modularized, requiring supervised training and a lot of training data and majorly lacking the robustness required in human-robot conversations. Zhao and Eskenazi (2016) presented an End-to-End RL approach to dialogue state tracking and policy learning in the DM where the agent asks the user a series of Yes/No questions to find the correct answer to guessing the famous person the user is thinking of. In the original paper, Li proposes a framework where DQN is used in the DM training to learn the interactive policy of the dialogue system, thereby improving flexibility in question types and robustness against noises in user utterances. Most importanly, the structure allows for user requests during dialogues.

This project is deeply inspired by the research done in the original paper, where they created TC-Bot, a Chatbot with an End-to-End learning framework for task-completion dialogue systems. The neural dialogue system directly interacts with a structured database, accesses information and accomplishing certain tasks. In "Training a Goal-Oriented Chatbot with Deep Reinforcement Learning", the author builds a basic pipeline for a rule-based user/agent dialogue with simplistic deep learning model. In this project we aim to develop further this rule-based dialogue system with improvements of the deep learning model, more specifically with Double Deep Q-Learning (DDQN) and Prioritized Experience Replay (PER).

## 2. Solution Concepts

###### 2.1 End-to-End Task-Completion Neural Dialogue Systems Framework

A brief overview of the structue of the original GO Dialogue System architectue is given.
<img src="fig/original.png" width = 500>
As listed before. there are 3 main components in the GO Dialogue System: The Dialogue Manager (DM), Natural Language Understanding (NLU) unit and Natural Language Generator (NLG) unit. The DM is split into the Dialogue State Tracker (DST) and the Policy Learner for the Agent itself. The Policy Learner is represented by a DQN. Furthermore, the system contains a User with a user goal which represents what the user's wishes to get out of the conversation.

Inside a dialogue system round, the user utters something which is then processed by the NLU component and outputs into representation known as a semantic frame. The DST inside the DM processes the semantic frame input received (e.g. request(moviename;genre=action; date=this weekend)) and performs the following:

1. Database Querying to retrieve available results
2. Updating of the DST based on results from the querying
3. Preparing state representation $s_t$ from the database query results and the latest user dialogue action for  the Agent's Policy Learning.

The DQN receives $s_t$ and action $a_t$ is outputed in the form of a semantic frame. i.e. $s^{\pi}_t \rightarrow a_t$ where $a_t$ is chosen according to $\pi(s_t)$ and $\pi(s_t)$ is optimized by the DQN. The Neural Network takes $s_t$ from the Dialogue State Tracker and outputs $Q(s_t, a; \theta)$ for all actions $a$. According to the original paper, the success rate are greatly improved with Experience Replay and the usage of target network in the DST.


The database can also be queried to add information to the Agent action such as movie ticket info. The Agent action is then processed by the NLG component which converts it to natural language for the user to read and understand.

**A simplified architecture and system for the GO Dialogue System implemented in this project is as follows:**

<img src="fig/flow.png" width = 500>

We follow the same Dialogue Manager system architecture in our implemented model with modification to the User Simulator. The NLG and NLU units are not included in the model in this project.

**User Simulator**

This is a solution to the problem of requiring heavy training with real users in previous Dialogue System research. By simulating an actual user using deterministic rules, the Agent can be trained much faster than if real users were used to give it multiple hours of training. Internal states are not only used to represent the goals and constraints of the User Simulator as well as keeping track of the conversation history inside an episode. The goal is picked randomly for each episode from a list of available user goals samples from a corpus where a goal consists of a set of constraints. Each round an action is crafted in response to the Agent's action using deterministic rules along with a few stochastic rules to create diversity in responses.

A user goal consists of inform slots and request slots. The inform slots of a user goal simulate constraints of a user in achieving its goal. The request slots simulate the user gaining information about what it doesn't know but wants to know from the agent in order to achieve its goal.

<blockquote> the user state $s_u$ is factored into an agenda $A$ and a goal $G$. The goal consists of constraints $C$ and request $R$. At each time-step $t$, the user simulator generates the next user action $a_{u,t}$ based on the current state $s_{u,t}$ and the last agent action $a_{m,t-1}$, and then updates the current status $s'_{u,t}$.
</blockquote>

A default slot “ticket” is added to the request slots of every goal. The Agent must successfully inform a value to fill the default slot to fulfill the goal and succeed in an episode.

The User Simulator Internal States keeps track of the goal slots and history of the current the conversation. It is used to create the user action each step. Specifically the state is a list of 4 dictionaries of slots and an intent:
1. Rest slots: All inform and request slots from the goal that are uninformed by either the agent or user yet
2. History slots: All inform slots from user and agent actions so far in the conversation episode
3. Request slots: Request slots that the user wants to request in immediate future actions
4. Inform slots: Inform slots that the agent intends to inform in the current action being crafted
5. Intent: The intent of the current action being crafted

**Error Model Controller (EMC)**

The EMC is used to add error to the User Simulator’s action at the semantic frame level. In the original paper, it was shown to improve significantly the results of training the policy on semantic frames. It is introduced to simulate noises from the LU component, and thus resulting in noisy communication between the User and the Agent in order to train the model robustness. They propose 2 forms of errors to be infused in the conversations: Slot Level error and Intent level error. 

*Types of slot level error for each inform slot in the action:*

- Replace the value with a random value for that key
- Replace the whole slot by exchanging for a random key and random value for that key
- Delete the slot entirely

The slot level error is infused is randomly chosen with equal probability for all 3 errors above for a slot.

*Types of intent level error for each inform slot in the action:*

- Replace intent with a random intent

###### 2.2 Deep Q-Network

By approximating the $Q$ function with a neural network, this network will take a state as an input and produce an estimation of the $Q$ function for each action. 

<img src="fig/dqn.png" width = 200>

In order to stabilize the $Q$ function the authors of the original paper emphasizes the importance of using (1) Target $Q$-Network and (2) Experience Replay in the DQN.

**Target Q-Network**

The $\epsilon$-greedy policy is used in training in order to encourage exploration. This policy behaves greedily most of the time, but chooses a random action with probability $\epsilon$. The estimated $Q$ function would then move towards the optimal $Q*$ function. As the DQN is learning the $Q*$ target function, this is an off-policy RL problem as our target neural network will be used to compute the loss for every action during training. In our case, the $Q*$ function is estimated by a separate netural network.

**Experience Replay**

During each simulation step, the DQN agent performs an action $a$ in state $s$, receives immediate reward $r$ and come to a new state $s’$. This results in the tuple $(s, a, r, s')$. As we are using a neural network, our estimation is now towards $Q(s,a) \rightarrow r+ \gamma max_aQ(s',a)$. The key idea of experience replay in the context of our DQN Agent is that at each epoch, we simulate $N$ episodes and we store the transitions $(s_t, a_t, r_t, s_{t+1})$ in our experience replay buffer for training. During an epoch, the current DQN will be updated based on the batch size and at the last epoch the $Q*$ network will be replaced by the current DQN.

Lastly, because our memory is finite, all experience tuples from the simulation are accumulated and thus flushed only when the current DQN agent reaches a success rate threshold, and then the current experience tuples from the current RL agent to used to re-fill the buffer. This is because the initial performance of the DQN Agent would not be strong enough to produce high-performing experience replay tuples. At every simulation epoch, we estimate the success rate of the current DQN Agent is estimated by running multiple dialogues with the Agent. If the current DQN Agent performs better than the target network $Q*$, the experience replay buffer would then be flushed.

###### 2.3 Double DQN
The Deep Reinforcement Learning with Double Q-learning paper reports that although Double DQN does not always improve performance, it substantially benefits the stability of learning. This improved stability directly translates to ability to learn the policy of our DQN Agent better.

In this neural network algorithm, 2 $Q$ functions $Q1$ and $Q2$ are independently learned. One function is then used to determine the maximization action and second to estimate the $Q*$ value. Either $Q1$ or $Q2$ is updated randomly with formulae:

$Q1(s,a)\rightarrow r + \gamma Q2(s',argmax_aQ1(s',a))$ 

or

$Q2(s,a)\rightarrow r + \gamma Q1(',argmax_a Q2(s',a))$

It was proven that by decoupling the maximizing action from its value in this way, one can indeed eliminate the maximization bias and thus improving the stability in learning. 

In our DQN Dialogue System Agent we already have two different networks giving us two different estimates i.e. $Q$ and our target network $Q*$, in order to achieve the DDQN algorithm we only need to modify the original target formula to

$Q(s,a)\rightarrow r + \gamma Q* (s',argmax_a Q(s',a))$

As seen from above, the behaviour network gives us the action that it ranks the highest. However, we let the target network have the evaluate the score of such an action. The difference between DQN and DDQN is in the calculation of the target Q-values of the next states. In DQN, we simply take the maximum of all the Q-values over all possible actions. This is likely to select over-estimated values, hence the researchers proposed to estimate the value of the chosen action instead. 

###### 2.4 Prioritized Experience Replay

As our DQN Agent learns from conversation episodes we would like to improve the accuracy by implementing a system where the DQN Agent learns from episodes that have failed i.e. the $Q*$ function differs greatly our target $Q*$. 

Prioritized Experience Replay (PER) is a strategy which helps improving learning by changing the sampling distribution. We prefer to learn from transitions that does not fit well to our current estimate of the $Q$ function as these are the transitions that we can learn most from. 

Given sample $S = (s, a, r, s')$ we define the distance between the $Q(s, a)$ and its target $T(S)$: 

$error=\mid Q(s,a) - T(S) \mid$

For our DDQN neural network described above, $T$ would be:

$T(S) = r + \gamma Q* (s',argmax_{a}Q(s',a))$

This error is stored in the Agent’s memory along with every iteration and is updated in each learning step.

**Proportional Prioritization**

The error is converted to *priority* :

$p=(error+\epsilon)^\alpha$

Epsilon $ϵ$ is a small positive constant that ensures no transition with zero priority. $\alpha$, ($0\leqslant \alpha \leqslant1$), controls the difference between high and low error. It determines how much prioritization is used. With $'alpha=0$ we would get the uniform case.

Priority is then translated to probability of being chosen for replay. A sample $i$ has a probability of being picked during the experience replay determined by a formula:

$P_i=p_i\sum_{k} p_k$

At each iteration we will get a batch of samples with this probability distribution and train our network on it. We only need an effective way of storing these priorities and sampling from them. For initialization, we fill the memory using random agent during warm-up. As the neural network untrained during warm-up it is likely to return a $Q$ value around zero for every input. In this case the error formula becomes very simple:

$error=\mid Q(s,a) - T(S)\mid =\mid Q(s,a)-r-\gamma Q* (s',argmax_aQ(s',a)) \mid=\mid r \mid $

The error during warm-up is therefore the reward experienced in a given sample.

To allow for efficient implementation in storing and selecting the prioritized experiences by changing the array data structure into a binary tree data structure whose parent's value would be the sum of its children; samples would then be stored as leaf nodes. For a value $s$, $0\leqslant s \leqslant \sum_{k} p_k$, we use the following retrieval algorithm recurively (pseudo code) to select the prioritized experiences:

```python
def retrieve(n, s):
  if n is leaf_node: return n

  if n.left.val >= s: return retrieve(n.left, s)
  else: return retrieve(n.right, s - n.left.val)
```


## 3. Choice and Description of Data
We consider a task-completion dialogue system for helping users book movie tickets. Over the course of conversations, the dialogue system gathers information about the customer’s desires and ultimately books the movie tickets. A binary outcome of success or failure is assessed at the end of the conversation, based on whether a movie is booked, and whether the movie satisfies the users constraints.

We use the same dataset provided in the source code of TC-Bot from the original paper. In total, we have labeled 280 dialogues, and the average number of turns per dialogue is approximately 11. The user goals are generated using a set of labeled conversational data. TC-Bot comes with the file of user goals of which we will be using.
This dataset includes three databases for different purposes: 

1. Movie Database for the DQN Agent to retreive information about the tickets and inform the User
2. Movie Dictionary as a dictionary of all input values of the details the tickets for the EMC
3. User Goals for training using the User Simulator

**Movie Database**

This is the database containing information about all 991 available movie tickets. The DQN Agent would query this database in order to provide inform slots to the User/User Simulator during a conversation episode. The database is stored in the format of a dictionary with key-value pairs of the movie ticket number and the details of the movie ticket. Details of the tickets would typically include the title, screening date and time of the movie; the city, zip code, area, state of the movie theatre, as well as the other miscellaneous details such as the food the theatre cafeteria serves and critic ratings.

In [4]:
import pickle
movie_db = pickle.load(open("data/movie_db.pkl", 'rb'), encoding='latin1')
print('Movie ticket number 81 :', movie_db[81])

Movie ticket number 81 : {'city': 'bellevue', 'theater': 'bellevue lincoln square cinemas', 'zip': '98101', 'distanceconstraints': 'area', 'state': 'wa', 'other': 'serves seafood', 'starttime': 'night', 'date': 'Friday', 'moviename': '10 cloverfield lane'}


**Movie Dictionary**

The dictionary contains all inputs about the movies collected from the original dialogue system dialogues. The keys are the different slots that can be in a ticket and the values are lists of the possible values for each slot. This dictionary is mainly used for generating nuanced state representations of the User Simulator through the Error Model Controller

In [6]:
movie_dict = pickle.load(open("data/movie_dict.pkl", 'rb'), encoding='latin1')
print('Movie ticket slots are :', movie_dict.keys())

Movie ticket slots are : dict_keys(['city', 'numberofpeople', 'theater', 'description', 'zip', 'numberofkids', 'distanceconstraints', 'critic_rating', 'price', 'greeting', 'actor', 'date', 'state', 'other', 'mpaa_rating', 'starttime', 'theater_chain', 'genre', 'video_format', 'moviename'])


Some examples of the values are:

In [13]:
print(movie_dict['city'][:5])
print(movie_dict['theater'][:5])
print(movie_dict['numberofpeople'][:5])
print(movie_dict['actor'][:])
print(movie_dict['starttime'][:5])
print(movie_dict['genre'][:5])
print(movie_dict['moviename'][:5])

['hamilton', 'manville', 'bridgewater', 'seattle', 'bellevue']
['manville 12 plex', 'amc dine-in theatres bridgewater 7', 'bridgewater', 'every single theatre', 'bellevue lincoln square cinemas']
['2', '5', 'two', '9', 'three']
['ryan reynolds', 'tina fey']
['10:30am', '11:10am', '1:10pm', '1:50pm', '3:45pm']
['comedy', 'comedies', 'kid', 'action', 'violence']
['zootopia', 'whiskey tango foxtrot', 'how to be single', 'kung fu panda 3', 'How to be single']


**User Goals**

This is a database of all 128 of the user requests/goals  as a list of dictionaries that contain request and inform slots of each goal. This database is for the DQN Agent to find a ticket that fits the user’s constraints which is given by that episode’s user goal. 

In [29]:
movie_user_goals = pickle.load(open("data/movie_user_goals.pkl", 'rb'), encoding='latin1')
print("One example:", movie_user_goals[2])
print("\n")
print("Example where the user directly requests slots from the Agent to inform him or her about:", movie_user_goals[22])

One example: {'request_slots': {}, 'diaact': 'request', 'inform_slots': {'city': 'birmingham', 'numberofpeople': '4', 'theater': 'carmike summit 16', 'state': 'al', 'starttime': 'around 6pm', 'date': 'today', 'moviename': 'deadpool'}}


Example where the user directly requests slots from the Agent to inform him or her about: {'request_slots': {'date': 'UNK', 'theater': 'UNK', 'starttime': 'UNK'}, 'diaact': 'request', 'inform_slots': {'numberofpeople': '4', 'moviename': 'zootopia'}}


## 4. Model Implementation

#### Environment

Step 1: 
The current state is passed as input to the Agent to get the predicted action

Step 2:
This action is then passed to the DST module so that the conversation history canbe updated.Database query information is addedto the agent action. 

Step 3:
The Agent's action is then passed to the User (Simulator), who then makes a rule-based response.

Step 4:
The User outputs an action which is passed to the EMC for error infusion

Step 5:
The user action and error are then passed to the DST to save the User(Simulator)’s action in the conversation history

Step 6:
The next state is passed to the Agent to complete the $(s, a, r, s')$ experience tuple


#### Action

Actions are encoded as dictionaries with keys being ’intent’, ’inform slots’, and ’request slots’. The ’intent’ field contains whether or not this action is to inform, request, give thanks which indicates a match is found and the Agent is successful, or terminate the conversation. The inform and request slots contain the information the Agent wishes to communicate to the User or vice versa. The list of possible keys are listed in ```configurations.py``` and their values in the database dictionary mentioned above. A request slot contains a key the sender wants to find a value for from the receiver. ‘UNK’ is the value which means “unknown” as the sender does not know what value works for this slot yet.


#### Artificial Intelligence Implementation

For the Deep Q-Learning Neural Network we implemented the neural network structure using ```TensorFlow 1.15.2```. Our neural network consists of a hidden layer of size 80 for estimating the $Q$-function with weights and bias initializers of ```tf.random_normal_initializer(0,0.1)``` and ```tf.constant_initializer(0.1)``` respectively.
We have 2 separate neural networks for $Q_{behaviour}$ and $Q_{target}$ and we train the sample through minibatch of batch size 16. The main difference (code wise) between the original DQN and the Double DQN is that the $Q$ function $r + \gamma max_a Q(s',a)$. In order to implement the Double DQN the only modification would be changing the update to the $Q_{target}$ function.
```python
q_target[batch_index,np.array(actions)] = np.array(rewards)+self.gamma*q_next[batch_index,np.argmax(q_eval,axis=1)]*(1-np.array(dones))
```

Finally for the Prioritized Experience Replay with DDQN, we draw prioritised samples using the TD error term as defined previously. Notice the  factor $\alpha$ is set to be 0.6 so that prioritization occurs but it is not absolute prioritization. This promotes some exploration in addition to the PER process. Furthermore, we applied Importance sampling to further stabilize the DDQN+PER algorithm. As we don’t have an exact function for the TD error based on all the possible states, actions and rewards in an environment, we cannot explicitly minimise the TD error. Instead, what we have are samples from the environment in the form of these experience tuples (states, actions, rewards) and through importance sampling we can minimise the expected value of the TD error without skewing the expected value calculation. We do this by applying weights to the TD error:

$w_i = \left( \frac{1}{N} \cdot \frac{1}{P(i)} \right)^\beta$

This weight value is multiplied by the absolute value between $Q$ and $Q_{target}$, which has the same effect as reducing the gradient step during training. The variable N refers to the number of experience tuples already stored in memory and $\beta$ is initialized 0.4 in our case and "annealed" with increments of 0.001 with each episode. With $\beta$ being brought closer to 1, this would decrease the weights for high priority samples. In the original paper by Google DeepMind, the authors argue that at the beginning of the training, the learning is chaotic and the bias caused by the prioritisation doesn’t matter much anyway. It is only towards the end of the training that this bias needs to be corrected, thus by bringing $\beta$ closer to 1 in later training episodes would decrease the weights for high priority samples and therefore corrects the bias more. 

As experience samples with a high priority will be sampled more frequently under PER, this weight value ensures that the learning is slowed for these samples. The higher the probability of the sampling, the lower this weight value will be. This ensures that the training is not exclusively prioritizing these higher probability samples and therefore acts to correct the aforementioned bias.

#### Reward function

We define the rewards as is a piece-wise constant function determined by the success of the Agent. If the Agent successfully provides the correct response to the User agent, it gets a positive reward as a function
of the maximum number of rounds of the User agent as shown below. Similarly, it will get a negative reward based on the same quantity if it fails to answer the question correctly. The use of the maxrounds in reward calculation promotes queries to be answered with greater urgency.

$  \begin{equation}
    X=
    \begin{cases}
      4 \times maxrounds, & \text{if}\ success=1 \\
      -maxrounds, & \text{otherwise}
    \end{cases}
  \end{equation}$
  
In the case of training in the pipeline, we set the maximum number of rounds to be 20. If the maximum number of rounds is reached without success or success is reached, the Agent finished the current training run and resets state to generate a new user action. Given this criteria, the maximum achievable average reward is 80.

#### Model Building and Training

We trained our DQN Agent on Google Colab. We trained all 3 DQN Agents namely the original DQN Agent, the DDQN Agent and the DDQN+PER Agent. The pipeline with the original Agent serves as the baseline for the RL component.
For the DQN architecture, we used 2 fully connected layers as defined in the previous subsection. Also, the hidden layer used Relu activation functions with a linear activation for the output layer. 

The hyperparameters are:

- learning rate : $5e^-3$
- batch size: $16$
- hidden layer size: $80$
- $\gamma$ : $0.9$ 
- $\min \epsilon$ : $0$ 
- $\max \epsilon$ : $0.01$ 
- $\lambda$ : $0.001$

We tested all the DQN models with higher epsilon values to promote exploration but found that a greedy policy i.e. $\epsilon \rightarrow 0$ is the best performing model.

Mean squared error was used as the loss, and the Adam optimizer was used to minimize this loss. All models were trained for 40,000 episodes with a 1000 episode warm-up to fill the experience replay memory.

Experience Replay is carried out with every training frequency reached. After each training frequency defined as 100, the Agent is trained on its memory of experience. If the success rate of the period is greater than or equal to the current best success rate and surpasses a predefined threshold of 0.3, then the memory is emptied for the Agent to learn from the newer and better model. The model weights are copied to the target model as with all DQNs. The model is then trained on the current memory buffer of experiences.


## 5. Model Evaluation

The performance of the original DQN Agent in the pipeline is considered to be the baseline of this model for the reinforcement learning component of this project. Performance is measured here in terms of average reward and average success rate.

For the baseline DQN Agent show a success rate in answering queries correctly of $\approx 0.72$ and an
average reward of $\approx 42.16$. The DDQN agent performs better with a success rate of $\approx 0.77$ and an average reward return of $\approx 47.80$. This improvement is to be expected as the DDQN model has better learning capacity than the original model. The DDQN+PER agent performs marginally better having a success rate of $\approx 0.79$ and an average reward of $\approx 50.03$

|                | DQN               |       DDQN       | DDQN+PER         |
|----------------|-------------------|:----------------:|------------------|
| Success Rate   | 0.72 $\pm$ 0.21   | 0.77 $\pm$ 0.06  | 0.79 $\pm$ 0.05  |
| Average Reward | 42.16 $\pm$ 23.83 | 47.80 $\pm$ 6.24 | 50.03 $\pm$ 6.00 |

We can conclude that the use of DDQN and PER reduces the standard deviation of both the success rates and average reward by observing that the DDQN has an improvement in stability by 74% from the original DQN Agent; with the DDQN+PER Agent the improvement is by 75%. We can appreciate fully how PER and DDQN help stabilize the DQN agent through plotting of the above sucess rates and average reward.
<img src="fig/sr.png" width = 400> <img src="fig/rdw.png" width = 400>

As we can see the success rates and average reward of the orinigal model behave erratically compare to the other 2 models. This is because of the maximization bias of the original DQN Agent and thus resulting in poor performance in learning newer and unseen episodes. 

## 6. Conclusion

In this project we have successfully implemented a Goal-Oriented Chatbot model using DQN. By training our Model on a Movie Ticket Conversation database we perform different improvements to the DQN neural network and compare their results against the stability, success rates and average rewards.

We discovered that the DQN Agent works best with a greedy policy where not much exploration is made during learning. This is because our User Generator agent follow a deterministic rule based diaglogue generation model; exploration in this context would make the dialogue rounds longer in order for the DQN Agent to reach the user's goal. Hence, it fails in reaching satisfactory success rates as the default maximum round is set to be 20 in our DQN Model in order to encourage speedy conversations. DDQN is supplemented to the DQN Agent and we found out that the use of a target neural network helps stabilize and thus increase the success rates of the original Agent. WIth the original DQN Agent as the benchmark, the improvement of the sucess rate and average reward is 107% and 113% respectively. More importantly, the improvement in stability measured in standard deviation of the success rate is around 70%. 

Prioritized Experience Replay is also used to supplement the Reinforcement Learning componenet in our model. We used Importance Sampling to sample episodes where greater errors were made in order to encourage the DQN Agent to learn from the failed episodes. This leads to promising improvements in terms of sucess rates and average rewards, and most importantly the stability of our Model. However, due to the small size of the dataset, we did not expect a significant improvement in performance. Therefore for furthering our DQN Model, we would like to test our model against other task-completing conversations such as booking flights and hotel and for restaurant reservations. 

A second limitation to our DQN Model is the assumption of a finite User action space for the User Simulator. The rule-based User Simulator was chosen for simpler training of the model. This can be remedied with generative component such as an RNN and LSTM. A neural network backed generative User Simulator would not only allow for more realistic scenarios to be generated, more importantly by creating more human-like User Simulators, it would allow for the DQN Agent to fully reach its potential in completing tasks. 

In conclusion, in order to achieve sucessful Goal-Oreiented Chatbot not only the Reinforcement Learning component of the DQN Agent requires further updating; also, the User Simulator component must be able to generate "learning-worthy" queries in order to create a robust and accurate DQN Agent. 

## 7. Next steps: direction of further research

An obvious major change for furthering our model is to switch to a larger dataset with natural language conversational queries and thus giving potential to implementing the entire TC-Bot architecture including the LU and NLG. We aim to depart from the rule-based dialogue system by extending the Chatbot system to include a Natural Language understand unit. One idea is that we can implement an LSTM based language understanding module for identifying user intents and extracting associated slots. Likewise, unlike other deterministic rule-based Chatbots which have scripted natural language responses to inform and request further information from the user, it would be interesting if our DQN Agent is capable of returning machine generated speech. Another much welcomed experiment would be to test the DQN Chatbot Agent with different conversational datasets. Overall, the main goal is to create an Agent which is representative of a human customer service represetative, and much work is needed to develop in this arena of Reinforcement Learning.

## References
- Machine Intelligence and Understanding Laboratory. User simulation for task-completion
dialogues. https://github.com/MiuLab/TC-Bot, 2017. [Accessed 25.04.2020]
- Max Brenner. Goal-oriented chatbot with deep rl. https://towardsdatascience.com/
training-a-goal-oriented-chatbot-with-deep-reinforcement-learning-parti-introduction-and-dce3af21d383, 2018. [Accessed 28.04.2020]
- Volodymyr Mnih, Koray Kavukcuoglu, David Silver, Alex Graves, Ioannis Antonoglou, Daan
Wierstra, and Martin Riedmiller. Playing atari with deep reinforcement learning. arXiv preprint
arXiv:1312.5602, 2013.
- Hado van Hasselt, Arthur Guez, and David Silver. Deep reinforcement learning with double
q-learning. CoRR, abs/1509.06461, 2015.
- Alexander I Rudnicky, Eric H Thayer, Paul C Constantinides, Chris Tchou, R Shern, Kevin A Lenzo, Wei
Xu, and Alice Oh. 1999. Creating natural dialogs in
the carnegie mellon communicator system. In Eurospeech.
- Tiancheng Zhao and Maxine Eskenazi. 2016. Towards end-to-end learning for dialog state tracking
and management using deep reinforcement learning.
In Proceedings of the 17th Annual Meeting of the
Special Interest Group on Discourse and Dialogue.
pages 1–10.
- Mnih et al. - Human-level control through deep reinforcement learning, Nature 518, 2015
- Hado van Hasselt - Double Q-learning, Advances in Neural Information Processing Systems, 2010 
- Schaul et al. - Prioritized Experience Replay, arXiv:1511.05952, 2015
- Jaroiru - Prioritized Experience Replay. jaromiru.com. https://jaromiru.com/2016/11/07/lets-make-a-dqn-double-learning-and-prioritized-experience-replay/#fn:1 [Accessed 1.05.2020]
- Peng, B., Li, X., Li, L., Gao, J., Celikyilmaz, A., Lee, S., & Wong, K.-F. (2017). Composite Task-Completion Dialogue Policy Learning via Hierarchical Deep Reinforcement Learning. Proceedings of the 2017 Conference on Empirical Methods in Natural Language Processing. doi: 10.18653/v1/d17-1237